## Wumpus world


### Student name: Mikael Olsson, Atichoke Nantarat
### DU email address: h20mikol@du.se, h20atina@du.se

Importing packages

In [13]:
%pylab inline
from IPython.display import clear_output
from random import randrange

Populating the interactive namespace from numpy and matplotlib


All code goes into this cell, we have a class called "WumpusGame" which has all the funtionality. It handles maze generation, displaying the maze, checking for status effects, mutating, creating solutions, resetting the maze and agent, moving the agent, making the agent shoot, evaluating solutions and the algorithm itself.

We decided to use the hillclimber algorithm to solve our maze since that's the one we used in lab 5.

All functionality is there but we decided to not show things like the "status bar" (breeze, stench etc) for peformance purposes. The lines to print the status effects are currently commented for performance (it writes so many lines it crashes vscode.)

In [14]:
class WumpusGame:
    def __init__(self,pitProb):
        #this is the constructor
        self.pitProb = pitProb
        self.breeze = False
        self.stench = False
        self.bump = False
        self.glitter = False
        self.score = 0
        self.wumpus = True
        self.agentX = 0
        self.agentY = 0
        self.agentAlive = True
        self.scores = []
        self.createMaze()       
        while not self.isSolvable():
            self.createMaze()    
              
    def createMaze(self):
        self.maze=list()
        self.maze_backup=list()
        
        for i in range(4):
            row=[]
            for j in range(4):
                row.append(".")
            self.maze.append(row)
            self.maze_backup.append(row)
        self.maze[self.agentX][self.agentY]='A'
        
        stuff=["W","G"]
        for s in stuff:
            x = numpy.random.choice([0,1,2,3])
            y = numpy.random.choice([0,1,2,3])
            while self.maze[x][y]!="." or self.maze[x][y] == 'A':
                x = numpy.random.choice([0,1,2,3])
                y = numpy.random.choice([0,1,2,3])
            self.maze[x][y]=s
            self.maze_backup[x][y]=s
        
        ky = -1
        for i in self.maze:
            kx = -1
            ky += 1
            for j in i:
                kx += 1
                if j == '.':
                    rnd = numpy.random.random()
                    if rnd < self.pitProb:
                        self.maze[kx][ky] = 'P'
        
    def showMaze(self):
        S=""
        for i in range(4):
            for j in range(4):
                S+=self.maze[i][j]
            S+="\n"
        print(S)
        
        
    def checkDebuff(self):
        xm=[0,1,0,-1]
        ym=[1,0,-1,0]
        for i in range(4):
            targetX=self.agentX+xm[i]
            targetY=self.agentY+ym[i]
            
            if (targetX >= 0 and targetX <=3) and (targetY >= 0 and targetY <=3):
                sense = self.maze[targetX][targetY]
                if sense =='P':
                    self.breeze = True
                if sense =='W':
                    self.stench = True              
            else:
                continue
        

    def resetDebuff(self):
        self.breeze = False
        self.stench = False
        self.glitter = False
        self.bump = False
    
    def makeMutant(self, solution, my):
        mutant = []
        for i in list(solution):
            if np.random.random()<my:
                rnd = randrange(0,4)
                mutant.append(rnd)
            else:
                mutant.append(i)
        return mutant
    
    def createSolution(self,maxSteps):
        self.maxSteps = maxSteps
        solution = []
        hasShot = False
        for i in range(maxSteps):
            step = numpy.random.choice([0,1,2,3,4,5,6,7])
            if step > 3 and hasShot == False:
                hasShot = True
                solution.append(step)
            else:
                step = numpy.random.choice([0,1,2,3])
                solution.append(step)       
        return solution      
    
    def isSolvable(self):        
        curX = 0
        curY = 0
        xm=[0,1,0,-1]
        ym=[1,0,-1,0]
        maxSteps = 1000000
        for i in range(maxSteps):
            step = numpy.random.choice([0,1,2,3])
            x = curX + xm[step]
            y = curY + ym[step]

            if x == -1:
                continue
            if x == 4:
                continue
            if y == -1:
                continue
            if y == 4:
                continue

            if self.maze[x][y] == '.' or 'G':
                curX = x
                curY = y
                if self.maze[x][y]=='G':
                    print("This maze has a valid solution.")
                    return True
        return False   
    
    def reset(self):
        self.maze = self.maze_backup
        self.agentX = 0
        self.agentY = 0
        self.maze[self.agentX][self.agentY]='A'
        self.breeze = False
        self.stench = False
        self.glitter = False
        self.bump = False
        self.wumpus = True
        score = self.score
        self.score = 0
        
        return score
    
    def resetAgent(self):
        self.agentAlive = True    
    
    def agentMove(self, step):
        self.score -= 1
        xm=[0,1,0,-1]
        ym=[1,0,-1,0]
        x = self.agentX + xm[step]
        y = self.agentY + ym[step]

        if (x >= 0 and x <=3) and (y >= 0 and y <=3):
            if self.maze[x][y] == '.':
                self.maze[self.agentX][self.agentY] = '.'
                self.agentX = x 
                self.agentY = y 
                self.maze[self.agentX][self.agentY] = 'A'

            if self.maze[x][y] == 'G':
                self.score += 1000
                score = self.reset()
                self.scores.append(score)
                # print(f'The agent found the gold and escaped with it! {score}')
                self.agentAlive = False
            
            if self.maze[x][y] == 'W':
                if self.wumpus == True:
                    self.score -= 1000
                    score = self.reset()
                    self.scores.append(score)
                    # print(f'The agent has died to the wumpus. {score}')
                    self.agentAlive = False
                    
            if self.maze[x][y] == 'P':
                self.score -= 1000
                score = self.reset()
                self.scores.append(score)
                # print(f'The agent has fallen into a pit. {score}')
                self.agentAlive = False     
        else:
            self.bump = True


    def agentShoot(self, direction):
        self.score -= 10
        xm=[0,1,0,-1]
        ym=[1,0,-1,0]
        shootingX = self.agentX
        shootingY = self.agentY
        
        for i in range(5):
            sx=shootingX+xm[direction]
            sy=shootingY+ym[direction]    
            if (sx >= 0 and sx <=3) and (sy >= 0 and sy <=3):      
                shootingX = sx
                shootingY = sy
                
                if self.maze[sx][sy] == 'W':
                    self.wumpus = False
                    # print('The screech of a dying creature echoes throguh the cave, the Wumpus has been defeated. ')
                else:
                    continue
            else:
                break

    
    def Agent(self, step, verbose):
        self.checkDebuff()
        if step >= 4:
            self.agentShoot(step-4)
        else:
            self.agentMove(step)
        
        if verbose == True:    
            if self.breeze == True or self.stench == True or self.glitter == True or self.bump == True :
                print(f'Breeze: {self.breeze} Stench: {self.stench} Glitter: {self.glitter} Bump: {self.bump}')

        
        self.resetDebuff()
        
        
    def resetScores(self):
        resetScores = []
        resetScores.clear()
        for i in self.scores:
            resetScores.append(i)
        self.scores.clear()
        return resetScores
        
    clear_output(wait = True)
    def hillclimberAlgorithm(self, maxSolutions, maxGenerations, maxSteps, my):
        solutions = []
        solutionsMutant = []
        
        i = 0
        while i != maxSolutions:
            solution = self.createSolution(maxSteps)
            solutions.append(solution)
            i += 1
        
        best_solution = self.createSolution(maxSteps)
       
        self.evaluateSolution(best_solution)
        scoru = self.resetScores()
        best_performer = scoru[0]
        
        generation = 0
        #Hillclimb
        while generation != maxGenerations:
            generation += 1
            
            for i in solutions:
                self.evaluateSolution(i)
            
            scores = self.resetScores()
            scores.append(best_performer)
            solutions.append(best_solution)
            
            #picks the best score as best_performer
            for i in scores:
                if i > best_performer:
                    best_performer = i
                    
            clear_output(wait = True)
            print(f'Generation: {generation}. Best performing solution had the score of {best_performer}')
            
  
            if best_performer > -5000:
                index = scores.index(best_performer)
            else:
                print('This generation was a failure')
                index = 10
            
            # print(f'Index is {index}')
            best_solution = solutions[index]
            solutions.clear()
            
            for i in range(maxSolutions):
                solutions.append(best_solution)
            
            for i in solutions:
                mutant = self.makeMutant(i, my)
                solutionsMutant.append(mutant)
            
            solutions.clear()
            for i in solutionsMutant:
                solutions.append(i)
            
            solutions.append(best_solution)
            solutionsMutant.clear()
            scores.clear()
        
        print(f'The algorithm has finished. Best score: {best_performer} had this solution {best_solution}')
                
    def evaluateSolution(self, solution):
        self.resetAgent()
        for i in solution:
            self.Agent(i,False)
            if self.agentAlive == False:
                break
        else:
            self.score -= 3555
            score = self.reset()
            self.scores.append(score)
            # print(f'The agent has collapsed from exhaustion. {score}')
            self.agentAlive = False

        
maze=WumpusGame(0.2)
maze.showMaze()

This maze has a valid solution.
A..G
P...
P...
...W



This is us running the code.

In [15]:
maze.hillclimberAlgorithm(maxSolutions=100,maxGenerations=1000,maxSteps=100,my=1)

Generation: 1000. Best performing solution had the score of 987
The algorithm has finished. Best score: 987 had this solution [5, 0, 0, 0, 2, 3, 2, 0, 1, 1, 1, 2, 3, 0, 2, 1, 0, 3, 0, 1, 0, 0, 1, 3, 2, 0, 0, 2, 1, 1, 2, 0, 3, 1, 2, 1, 1, 3, 1, 0, 2, 2, 2, 3, 0, 3, 1, 3, 1, 0, 1, 0, 1, 1, 2, 1, 1, 1, 0, 2, 0, 2, 0, 0, 2, 1, 0, 1, 0, 1, 2, 0, 0, 2, 3, 2, 0, 1, 3, 2, 1, 3, 2, 1, 0, 3, 1, 0, 2, 1, 2, 2, 2, 3, 3, 2, 0, 2, 2, 1]
